###CHatbot

In [1]:
#Kutubxonalarni o'rnatish
import tensorflow as tf 
import numpy as np
import pandas as pd 
import json
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, GlobalMaxPooling1D,Flatten
import matplotlib.pyplot as plt 

In [2]:
#json fileni yaratish

%%writefile content.json
{
  "intents": [
    {
      "tag": "greetings",
      "input": ["hello", "hi", "hello"],
      "responses": ["hi", "welcome to here", "hey yo, bye bye"]
    },
    {
      "tag": "googbye",
      "input": ["bye", "googbye", "bye bye"],
      "responses": ["bye", "wellcome, to here", "hey yo, nice to meet you"]
    }
  ]
}

Writing content.json


In [3]:
#importing json data
with open('/content/content.json') as content:
  data1= json.load(content)

In [4]:
#malumotlarni dataframega keltirish
tags = []
inputs = []
responses = {}
for intent in data1['intents']:
  responses[intent['tag']] = intent['responses']
  for lines in intent['input']:
    inputs.append(lines)
    tags.append(intent['tag'])

In [5]:
#converting to dataframe
data = pd.DataFrame({'inputs':inputs,"tags":tags})
data

,inputs,tags
0,hello,greetings
1,hi,greetings
2,hello,greetings
3,bye,googbye
4,googbye,googbye
5,bye bye,googbye


##Preprocessing
####converting to lowercase, encoding, encoding, textual data to numberical data

In [6]:
import string
data['inputs'] = data['inputs'].apply(lambda wrd: [w.lower() for w in wrd if w not in string.punctuation])
data['inputs'] = data['inputs'].apply(lambda wrd: ''.join(wrd))

data

,inputs,tags
0,hello,greetings
1,hi,greetings
2,hello,greetings
3,bye,googbye
4,googbye,googbye
5,bye bye,googbye


In [7]:

from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['inputs'])
train = tokenizer.texts_to_sequences(data['inputs'])
#apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)

#encoding the output
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])

In [8]:
input_shape = x_train.shape[1]
print(input_shape)

2


In [9]:
#defining vocabulary 
vocabulary = len(tokenizer.word_index)
print('word index: ',vocabulary)
outputLength = le.classes_.shape[0]
print('output length: ', outputLength)

word index:  4
output length:  2


In [10]:
#creating model
from keras.models import Model

i = Input(shape=(input_shape,))
x = Embedding(vocabulary+1,10)(i)
x = LSTM(10,return_sequences=True)(x)
x= Flatten()(x)
x = Dense(outputLength,activation="softmax")(x)
model = Model(i,x)
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [11]:
#modelni train qildirish
train = model.fit(x_train,y_train,epochs=200)

Epoch 1/200
1/1 [==============================] - 4s 4s/step - loss: 0.6911 - accuracy: 0.8333
Epoch 2/200
1/1 [==============================] - 0s 12ms/step - loss: 0.6907 - accuracy: 1.0000
Epoch 3/200
1/1 [==============================] - 0s 14ms/step - loss: 0.6902 - accuracy: 1.0000
Epoch 4/200
1/1 [==============================] - 0s 15ms/step - loss: 0.6898 - accuracy: 1.0000
Epoch 5/200
1/1 [==============================] - 0s 12ms/step - loss: 0.6893 - accuracy: 1.0000
Epoch 6/200
1/1 [==============================] - 0s 14ms/step - loss: 0.6888 - accuracy: 1.0000
Epoch 7/200
1/1 [==============================] - 0s 12ms/step - loss: 0.6883 - accuracy: 1.0000
Epoch 8/200
1/1 [==============================] - 0s 11ms/step - loss: 0.6878 - accuracy: 1.0000
Epoch 9/200
1/1 [==============================] - 0s 14ms/step - loss: 0.6873 - accuracy: 1.0000
Epoch 10/200
1/1 [==============================] - 0s 15ms/step - loss: 0.6868 - accuracy: 1.0000
Epoch 11/200
1/1 [===

In [12]:
import random

while True:
  texts_p = []
  prediction_input = input('You : ')

  #removing punct, converting to lower
  prediction_input = [letters.lower() for letters in prediction_input if letters in string.punctuation]
  prediction_input = ''.join(prediction_input)
  
  #Tokenizing and padding
  prediction_input = tokenizer.texts_to_sequences(texts_p)
  prediction_input = np.array(prediction_input).reshape(-1)
  prediction_input = pad_sequences([prediction_input],input_shape)

  #getting output
  output = model.predict(prediction_input)
  output = output.argmax()

  #finding the right tag and predicting
  response_tag = le.inverse_transform([output])[0]
  print('Robot : ',random.choice(responses[response_tag]))

  if response_tag == 'goodbye':
    break

You : hey
1/1 [==============================] - 1s 681ms/step
Robot :  hi
You : hoy
1/1 [==============================] - 0s 21ms/step
Robot :  hey yo, bye bye
You : goodbye
1/1 [==============================] - 0s 23ms/step
Robot :  hi


KeyboardInterrupt: ignored